# Chapter 8 — Handoffs (Task Delegation) 
## 01 Introduction

**Handoffs** let one agent delegate a task to **another, more specialized agent**. Think of it as saying, “You’ve got this,” and passing the ball—without losing the overall play. In many apps (support, research, coding), different agents excel at different jobs; handoffs make that teamwork explicit. (No capes required.)

## What it is
- A **built-in delegation mechanism** in the Agents SDK.
- To the model, a handoff looks like a **tool** (e.g., handing off to `Refund Agent` exposes a tool named `transfer_to_refund_agent`).
- You attach handoffs via an agent’s `handoffs=[...]` list—either to raw `Agent` objects or to configurable `Handoff(...)` descriptors.

## Why use it
- **Separation of concerns:** keep prompts small and roles crisp.
- **Better accuracy:** route math to the math agent, policy to the policy agent, etc.
- **Composable workflows:** a triage agent can pick the right specialist per query.

## How it feels at runtime
1. The current agent reasons about the user query.
2. It decides a specialist should handle it.
3. It invokes a **handoff tool**, transferring control (and conversation context) to the target agent.
4. The target agent responds and optionally hands back—or delegates further.

## Handoff vs. Agent-as-Tool
- **Handoff:** target agent takes control of the next turn(s). Great when the specialist should “own” the conversation for a bit.
- **Agent-as-Tool:** caller stays in charge, **invokes** a specialist and **integrates** the result in the same turn.

## Stability & observability (tiny checklist)
- Add explicit routing hints (e.g., `handoff_description`) so triage is predictable.
- Keep temperature low for the router agent.
- Log handoffs with hooks (`on_handoff`) so you **know** when delegation happened.
- Limit recursion depth—no infinite “please hold…” loops.

> TL;DR: Handoffs are your orchestration glue—cleanly delegate to the right specialist, keep roles tidy, and make your multi-agent system feel like a well-run team (with just a hint of office banter).


## 02. Basic usage — a simple handoff

Below we create two specialists (History / Math) and a **triage** agent that decides whom to delegate to.  
In the SDK, each handoff appears to the model as a **tool** (e.g., “transfer_to_history_tutor”), so the triage agent can pick one cleanly.

In [ ]:
from agents import Agent, Runner, set_tracing_disabled
import asyncio
from agents.extensions.models.litellm_model import LitellmModel
import os
from dotenv import load_dotenv

# --- Env & model ---
load_dotenv()
api_key = os.getenv('API_KEY')

base_url = "https://api.openai.com/v1"  
chat_model = "gpt-4.1-nano-2025-04-14"  
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


# --- Specialists ---
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

# --- Router / Triage ---
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=llm,
)


result = await Runner.run(triage_agent, "What is the capital of France?")
print(result.final_output)


The capital of France is Paris. Paris is not only the political and administrative center of the country but also a major cultural, economic, and historical hub. It has a rich history dating back to ancient times, known for iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris has played a significant role in numerous historical events, including the French Revolution and the development of modern art and philosophy.


In [2]:
from agents import Agent, Runner, handoff, RunContextWrapper, set_tracing_disabled

def on_handoff(ctx: RunContextWrapper[None]):
    print("Handoff called")


history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)


handoff_obj = handoff(
    agent=math_tutor_agent,
    on_handoff=on_handoff,
    tool_name_override="custom_handoff_tool",
    tool_description_override="Custom description",
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, handoff_obj],
    model=llm,
)


result = await Runner.run(triage_agent, "What is the capital of France?")
print(result.final_output)


The capital of France is Paris. Paris has been a significant cultural, political, and economic center for centuries, known for its historical landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It has played a crucial role in events like the French Revolution, the Enlightenment, and continues to be influential in global affairs today.


## 03. Customizing handoffs with `handoff(...)`

Besides passing `handoffs=[history_tutor_agent, math_tutor_agent]`, you can build a **custom handoff** using `handoff(...)`.  
This lets you override the **tool name/description**, attach an **immediate callback** when delegation happens, and (optionally) enforce **input typing/filtering** for the target agent.


In [8]:
from agents import Agent, Runner, handoff, RunContextWrapper, set_tracing_disabled

def on_handoff(ctx: RunContextWrapper[None]):
    print("Handoff called")

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

# Build a custom handoff entry that points to Math Tutor
handoff_obj = handoff(
    agent=math_tutor_agent,
    on_handoff=on_handoff,                       # fires immediately when delegation is chosen
    tool_name_override="custom_handoff_tool",    # default would be transfer_to_math_tutor
    tool_description_override="Custom description",
    # input_type=...,                            # (optional) expected input schema/type for the target agent
    # input_filter=lambda x: x.strip(),          # (optional) mutate/clean the input before passing on
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question, Always hand off via the provided tool",
    handoffs=[history_tutor_agent, handoff_obj],  # one plain Agent, one custom handoff
    model=llm,
)

result = await Runner.run(triage_agent, "What is 12 * 7")
print(result.final_output)

Handoff called
Let's find the product of 12 and 7.

To do this, we'll multiply 12 by 7:

12 × 7

One way to think about this is to break down 12 into 10 and 2:

(10 + 2) × 7

Now, use the distributive property:

(10 × 7) + (2 × 7)

Calculate each part:

10 × 7 = 70

2 × 7 = 14

Now, add these results together:

70 + 14 = 84

So, 12 multiplied by 7 equals **84**.


## 04. Handoff input — controlling *what* gets passed

Sometimes you want the model to **include specific data at delegation time** (e.g., a *reason for escalation*), or you want to **sanitize/reshape** the payload sent to the target agent. The SDK lets you do this via `handoff(..., input_filter=..., input_type=...)`.

In [10]:
from agents import Agent, Runner, handoff, RunContextWrapper, set_tracing_disabled, function_tool
from agents.extensions import handoff_filters

@function_tool
def get_weather(city: str) -> str:
    """Fetch the weather for a given city.

    Args:
        city: The city to fetch the weather for.
    """
    print("Weather tool used")
    return f"The weather in {city} is sunny"


history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)


# Customize the handoff: strip tool traces before passing input to Math Tutor
handoff_obj = handoff(
    agent=math_tutor_agent,
    input_filter=handoff_filters.remove_all_tools,
)

triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "Before deciding whom to delegate to, you MUST call `get_weather` with the relevant CITY inferred "
        "from the question (e.g., for 'capital of France', use 'Paris'). "
        "If the weather text contains 'sunny', delegate to Math Tutor; otherwise delegate to History Tutor."
    ),
    handoffs=[history_tutor_agent, handoff_obj],
    tools=[get_weather],
    model=llm,
)

result = await Runner.run(triage_agent, "What is the capital of France?")
print(result.final_output)

Weather tool used
The capital of France is Paris. Paris is not only the largest city in France but also a major cultural, political, and economic center. If you're interested in more details about Paris or related topics, feel free to ask!


## 05. Recommended routing prompt (handoff-friendly)

To help the model **understand and respect handoffs**, prepend OpenAI’s suggested system text to your own prompt. The SDK exposes it as:

In [5]:
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
print(RECOMMENDED_PROMPT_PREFIX)

# System context
You are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.



How to use (manual prepend)

In [ ]:
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions

billing_agent = Agent(
    name="Billing agent",
    instructions=prompt_with_handoff_instructions(
        "You handle billing questions. If the query concerns refunds, hand off to Refund Agent. Be concise."
    ),
    model=llm,
)
